In [1]:
import pandas  as pd
import numpy as np
import plotly.express as px

import yfinance as yf

import datetime

pd.__version__

'1.5.3'

In [2]:
def custom_cumsum(df):
    sol=pd.DataFrame()

    ticket=df["Ticket"].values[0]

    sol["Ticket Name"]=ticket
    sol["Date"]=df["Date"]
    sol["Tickets"]=df["cantidadTickets signo"].cumsum()

    start_date=sol["Date"].min()
    start_date=datetime.date(start_date)

    end_date=datetime.date(datetime.today())
    end_date
    idx = pd.period_range(min(df.date), max(df.date))
    sol.reindex(idx, fill_value=0)
    stock=yf.Tickers(ticket)
    dividend_data=stock.history(start=start_date,end=end_date)["Dividends"].replace(to_replace=0,method="bfill")

    sol=sol.merge(stock)


    return sol

In [3]:
path=r"G:\Mi unidad\Personal\Finanzas Personales\Ingresos.xlsx"

In [4]:
from datetime import datetime
movimientos_cartera=pd.read_excel(path,sheet_name="Movimientos Cartera")

In [5]:
movimientos_cartera.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  60 non-null     datetime64[ns]
 1   Ticket                60 non-null     object        
 2   cantidadTickets       60 non-null     int64         
 3   precioMovimiento      60 non-null     object        
 4   gastoLocal            60 non-null     object        
 5   gasto                 51 non-null     float64       
 6   tipoMovimiento        60 non-null     object        
 7   tipoCartera           60 non-null     object        
 8   Comision Transaccion  60 non-null     float64       
 9   Comision AutoFx       27 non-null     float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 4.8+ KB


# Preprocesado

In [6]:
movimientos_cartera["cantidadTickets signo"]=movimientos_cartera.apply(lambda x: x["cantidadTickets"] if x["tipoMovimiento"]=="Compra" else -x["cantidadTickets"],axis=1)
movimientos_cartera.head(1)



,Date,Ticket,cantidadTickets,precioMovimiento,gastoLocal,gasto,tipoMovimiento,tipoCartera,Comision Transaccion,Comision AutoFx,cantidadTickets signo
0,2021-08-20,BASE,2,"$37,45","$74,90",64.13,Compra,Growth,0.5,NaN,2


In [7]:
def agregate_operations(df):
    sol=pd.DataFrame()
    gasto_total=df["gasto"].sum()
    sol["Date"]=[df["Date"].values[0]]
    sol["Ticket"]=[df["Ticket"].values[0]]
    sol["cantidadTickets signo"]=[df["cantidadTickets signo"].values[0]]
    sol["gasto"]=[gasto_total]

    return sol

movimientos_cartera=movimientos_cartera.groupby(["Date"], group_keys=False).apply(agregate_operations)

movimientos_cartera.head()

,Date,Ticket,cantidadTickets signo,gasto
0,2021-08-20,BASE,2,64.13
0,2021-08-26,BASE,2,74.22
0,2021-08-31,AMD,1,255.27
0,2021-09-01,AMD,2,187.23
0,2021-09-02,AMD,-3,276.70


# Variacion Tickets

In [37]:
def custom_cumsum(df):
    sol=pd.DataFrame()

    ticket=df["Ticket"].values[0]
    sol["Ticket Name"]=df["Ticket"]
    sol["Date"]=df["Date"]
    sol["Tickets"]=df["cantidadTickets signo"].cumsum()

    start_date=sol["Date"].min()
    start_date=datetime.date(start_date)

    end_date=datetime.date(datetime.today())

    sol.index=sol["Date"]


    idx = pd.date_range(start_date,end_date)
    sol=sol.reindex(idx)
    sol=sol.drop("Date",axis=1)
    sol=sol.reset_index(names="Date")
    sol=sol.fillna(method="bfill").fillna(method="ffill")
    sol["Date"]=sol["Date"].apply(lambda x: datetime(x.year,x.month,x.day))
    stock=yf.Ticker(ticket)
    dividend_data=stock.history(start=start_date,end=end_date)["Dividends"].to_frame().reset_index(names="Date")
    dividend_data["Date"]=dividend_data["Date"].apply(lambda x: datetime(pd.Timestamp(x).year,pd.Timestamp(x).month,pd.Timestamp(x).day))

    

    sol=pd.merge(sol,dividend_data,on="Date")

    return sol


acumulated_data=movimientos_cartera.groupby(["Ticket"],as_index=False).apply(custom_cumsum)
#acumulated_data=acumulated_data[acumulated_data["Tickets"]>0]
acumulated_data.tail(30)

Date Ticket Name  Tickets  Dividends
14 199 2023-02-15          UL     18.0      0.000
   200 2023-02-16          UL     18.0      0.000
   201 2023-02-17          UL     18.0      0.000
   202 2023-02-21          UL     18.0      0.000
   203 2023-02-22          UL     18.0      0.000
   204 2023-02-23          UL     18.0      0.457
   205 2023-02-24          UL     18.0      0.000
   206 2023-02-27          UL     18.0      0.000
   207 2023-02-28          UL     18.0      0.000
   208 2023-03-01          UL     18.0      0.000
   209 2023-03-02          UL     18.0      0.000
   210 2023-03-03          UL     18.0      0.000
   211 2023-03-06          UL     18.0      0.000
   212 2023-03-07          UL     18.0      0.000
   213 2023-03-08          UL     18.0      0.000
   214 2023-03-09          UL     18.0      0.000
   215 2023-03-10          UL     18.0      0.000
   216 2023-03-13          UL     18.0      0.000
   217 2023-03-14          UL     18.0      0.000
   218 2023-03-15          UL     18.0      0.000
   219 2023-03-16          UL     18.0      0.000
   220 2023-03-17          UL     18.0      0.000
   221 2023-03-20          UL     18.0      0.000
   222 2023-03-21          UL     18.0      0.000
   223 2023-03-22          UL     18.0      0.000
   224 2023-03-23          UL     18.0      0.000
   225 2023-03-24          UL     18.0      0.000
   226 2023-03-27          UL     18.0      0.000
   227 2023-03-28          UL     18.0      0.000
   228 2023-03-29          UL     18.0      0.000

In [50]:
acumulated_data["month"]=acumulated_data["Date"].apply(lambda x: x.month)
acumulated_data["year"]=acumulated_data["Date"].apply(lambda x: str(x.year))
acumulated_data["dividend earnings"]=acumulated_data["Tickets"]*acumulated_data["Dividends"]

In [51]:
grouped_data=acumulated_data.groupby(["year","month"],as_index=False).sum()
grouped_data

C:\Users\afern\AppData\Local\Temp\ipykernel_7600\4041006905.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,year,month,Tickets,Dividends,dividend earnings
0,2021,8,31.0,0.000,0.000
1,2021,9,88.0,0.000,0.000
2,2021,10,84.0,0.000,0.000
3,2021,11,92.0,0.000,0.000
4,2021,12,113.0,0.000,0.000
5,2022,1,100.0,0.000,0.000
6,2022,2,95.0,0.000,0.000
7,2022,3,115.0,0.000,0.000
8,2022,4,107.0,0.000,0.000
9,2022,5,537.0,1.586,11.598


In [54]:
px.histogram(grouped_data,x="year",y="dividend earnings")

In [52]:
px.bar(grouped_data,x="month",y="dividend earnings",color="year")

In [36]:
movimientos_cartera.tail()

,Date,Ticket,cantidadTickets signo,gasto
0,2023-01-19,MAIN,14,504.40
0,2023-02-01,JNJ,3,449.16
0,2023-02-16,JNJ,4,594.12
0,2023-03-01,O,9,534.57
0,2023-03-14,TROW,5,498.37


In [10]:
acumulated_data=movimientos_cartera.groupby(["Ticket"],as_index=False).apply(custom_cumsum)
acumulated_data=acumulated_data[acumulated_data["Tickets"]>0]
acumulated_data

           Ticket Name  Tickets
2022-06-06           0        0
2022-06-07           0        0
2022-06-08           0        0
2022-06-09           0        0
2022-06-10           0        0
...                ...      ...
2023-03-22           0        0
2023-03-23           0        0
2023-03-24           0        0
2023-03-25           0        0
2023-03-26           0        0

[294 rows x 2 columns]


ValueError: You are trying to merge on period[D] and datetime64[ns, America/New_York] columns. If you wish to proceed you should use pd.concat

datetime.date(2023, 3, 25)

In [55]:
tickets=acumulated_data["Ticket Name"].unique()

# Dividendos

In [83]:
stock=yf.Ticker("ABBV")
stock.history(start=start_date,end=today_date)["Dividends"].replace(to_replace=0,method="bfill").to_frame()

,Dividends
Date,
2021-08-19,1.3
2021-08-20,1.3
2021-08-23,1.3
2021-08-24,1.3
2021-08-25,1.3
...,...
2023-03-20,0.0
2023-03-21,0.0
2023-03-22,0.0


In [59]:
data["Año"]=data["Ex-Date"].apply(lambda x: int(x.year))
data["Mes"]=data["Ex-Date"].apply(lambda x: (int(x.month))
data["Ingreso Bruto"]=data["Ingreso Bruto"].apply(lambda x: float(x[1:].replace(",",".")))
data["Año"]=data["Año"].apply(lambda x: str(x))


In [61]:
px.histogram(data,x="Mes",y="Ingreso Bruto",color="Año",barmode="group",nbins=12)

In [62]:
px.histogram(data,x="Año",y="Ingreso Bruto")